In [1]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
from tqdm import tqdm
import time
import random
import os

# Notes

### Offensive vs Defensive Styles

1. Man-to-Man Defense vs. Pass-Heavy Offense:
Strengths: Man coverage can effectively neutralize receivers, forcing the quarterback to make precise throws. It can also disrupt timing-based passing attacks and limit big plays.
Weaknesses: If the pass rush is not effective, the defense may struggle to cover receivers for an extended period, leading to potential big gains for the offense.


2. Zone Defense vs. Spread Offense:
Strengths: Zone coverage can provide better overall field coverage, making it difficult for the offense to exploit specific matchups. It can also force the quarterback to make difficult reads.
Weaknesses: Against a spread offense with quick, precise passes, zone coverage may leave gaps in the defense, allowing the offense to find open receivers easily.


3. Blitzing Defense vs. Strong Pocket Passer:
Strengths: Blitzing can put significant pressure on the quarterback, forcing quicker throws and disrupting the passing game.
Weaknesses: A strong pocket passer may exploit the blitz by quickly identifying and targeting hot routes or by releasing the ball before the pressure arrives, potentially leading to big plays.


4. Conservative Defense vs. Run-Heavy Offense:
Strengths: A conservative defense can focus on stopping the run and limiting big plays. It can also maintain gap discipline, making it harder for the offense to gain significant yardage on the ground.
Weaknesses: Against a potent run-heavy offense, a conservative defense may struggle to consistently stop the run and may get worn down over time.


5. Play-Action Offense vs. Aggressive Defense:
Strengths: An offense that effectively uses play-action can catch aggressive defenders out of position, creating opportunities for big plays downfield.
Weaknesses: An aggressive defense that bites on play-action fakes can leave open passing lanes and vulnerable to deep passes.


6. Power Running Offense vs. Blitzing Defense:
Strengths: A power running offense can wear down a blitzing defense and exploit the gaps created by defenders leaving their positions.
Weaknesses: Blitzing defenders can penetrate the backfield quickly and disrupt the running game, leading to tackles for loss or minimal gains.


7. West Coast Offense vs. Man Coverage Defense:
Strengths: The West Coast offense's quick, short passing game can create separation from man coverage defenders and lead to yards after the catch.
Weaknesses: If the man coverage defenders are physical and jam receivers at the line of scrimmage, it can disrupt the timing of West Coast plays.

# Code

In [ ]:
afc_head_coaches = [
    {"team": "Baltimore Ravens", "coach": "John Harbaugh", "style": "Harbaugh's teams are known for their physicality and toughness. On offense, they've recently been known for a unique, run-heavy attack led by QB Lamar Jackson. On defense, they're traditionally aggressive, often blitzing to generate pressure."},
    {"team": "Buffalo Bills", "coach": "Sean McDermott", "style": "McDermott is a defensive-minded coach. His defenses are typically disciplined and fundamentally sound, often playing a lot of zone coverage. On offense, his teams have been balanced, with a recent emphasis on the passing game with QB Josh Allen."},
    {"team": "Cincinnati Bengals", "coach": "Zac Taylor", "style": "Taylor is an offensive-minded coach with a background in the Sean McVay coaching tree. His offenses typically feature a lot of pre-snap motion and play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Cleveland Browns", "coach": "Kevin Stefanski", "style": "Stefanski is known for his run-heavy offensive scheme that features a lot of two tight end sets and play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Denver Broncos", "coach": "Sean Payton", "style": "Payton is another offensive-minded coach known for his aggressive play-calling. His offenses often feature a strong passing attack with a lot of variety in formations and personnel groupings. On defense, his teams have typically been aggressive, often blitzing to generate pressure."},
    {"team": "Houston Texans", "coach": "DeMeco Ryans", "style": "Ryans, a former player, is a defensive-minded coach. His defenses are typically aggressive, often blitzing to generate pressure. On offense, his teams have been more conservative, often focusing on the running game."},
    {"team": "Indianapolis Colts", "coach": "Shane Steichen", "style": "Steichen is an offensive-minded coach known for his work with quarterbacks. His offenses typically feature a balanced attack with a lot of play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Jacksonville Jaguars", "coach": "Doug Pederson", "style": "Pederson is an offensive-minded coach known for his aggressive play-calling. His offenses often feature a strong passing attack with a lot of variety in formations and personnel groupings. On defense, his teams have typically been aggressive, often blitzing to generate pressure."},
    {"team": "Kansas City Chiefs", "coach": "Andy Reid", "style": "Reid is an offensive-minded coach known for his innovative and aggressive play-calling. His offenses often feature a lot of pre-snap motion and misdirection to create favorable matchups. On defense, his teams have typically played a lot of zone coverage."},
    {"team": "Las Vegas Raiders", "coach": "Josh McDaniels", "style": "McDaniels is an offensive-minded coach known for his innovative play-calling. His offenses often feature a strong passing attack with a lot of pre-snap motion and misdirection. On defense, his teams have typically been more conservative, often playing a lot of zone coverage."},
    {"team": "Los Angeles Chargers", "coach": "Brandon Staley", "style": "Staley is a defensive-minded coach known for his innovative schemes. His defenses often feature a lot of pre-snap movement and disguise. On offense, his teams have been more balanced, with a recent emphasis on the passing game with QB Justin Herbert."},
    {"team": "Miami Dolphins", "coach": "Mike McDaniel", "style": "McDaniel is an offensive-minded coach known for his work with the running game. His offenses often feature a lot of pre-snap motion and misdirection. On defense, his teams have typically been more conservative, often playing a lot of zone coverage."},
    {"team": "New England Patriots", "coach": "Bill Belichick", "style": "Belichick is known for his defensive mind and his ability to adapt his game plan based on the opponent. He's known for taking away an opponent's best weapon and forcing them to play left-handed. On offense, his teams have been known to use a balanced attack, with a strong emphasis on the short passing game."},
    {"team": "New York Jets", "coach": "Robert Saleh", "style": "Saleh is a defensive-minded coach known for his energy and motivational skills. His defenses are typically aggressive, often blitzing to generate pressure. On offense, his teams have been more conservative, often focusing on the running game."},
    {"team": "Pittsburgh Steelers", "coach": "Mike Tomlin", "style": "Tomlin's teams are known for their physicality and discipline. On offense, they typically feature a balanced attack with a strong emphasis on the running game. On defense, they are known for their aggressive, blitz-heavy schemes."},
    {"team": "Tennessee Titans", "coach": "Mike Vrabel", "style": "Vrabel's teams are known for their physicality and discipline. On offense, they typically feature a run-heavy attack. On defense, they are known for their aggressive, blitz-heavy schemes."}
]

nfc_head_coaches = [
    {"team": "Arizona Cardinals", "coach": "Jonathan Gannon", "style": "Gannon is a defensive-minded coach known for his adaptability and player development skills. His defenses often feature a lot of pre-snap movement and disguise. On offense, his teams have been more balanced."},
    {"team": "Atlanta Falcons", "coach": "Arthur Smith", "style": "Smith is known for his offensive strategies, particularly his run-heavy approach. His offenses often feature a lot of pre-snap motion and play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Carolina Panthers", "coach": "Frank Reich", "style": "Reich is an offensive-minded coach known for his work with quarterbacks. His offenses typically feature a balanced attack with a lot of play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Chicago Bears", "coach": "Matt Eberflus", "style": "Eberflus is a defensive-minded coach known for his emphasis on effort and hustle. His defenses are typically disciplined and fundamentally sound, often playing a lot of zone coverage. On offense, his teams have been more conservative, often focusing on the running game."},
    {"team": "Dallas Cowboys", "coach": "Mike McCarthy", "style": "McCarthy is an offensive-minded coach known for his work with quarterbacks. His offenses typically feature a balanced attack with a lot of play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Detroit Lions", "coach": "Dan Campbell", "style": "Campbell is known for his motivational skills and player-centric approach. His teams are typically physical and tough, with a focus on the running game on offense. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Green Bay Packers", "coach": "Matt LaFleur", "style": "LaFleur is an offensive-minded coach known for his innovative play-calling. His offenses often feature a lot of pre-snap motion and play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Los Angeles Rams", "coach": "Sean McVay", "style": "McVay is an offensive guru known for his innovative play-calling and use of pre-snap motion. His offenses often feature a strong passing attack with a lot of play-action. On defense, his teams have typically played a lot of zone coverage."},
    {"team": "Minnesota Vikings", "coach": "Kevin O’Connell", "style": "O'Connell is known for his work with quarterbacks and his offensive strategies. His offenses typically feature a balanced attack with a lot of play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "New Orleans Saints", "coach": "Dennis Allen", "style": "Allen is a defensive-minded coach known for his attention to detail and player development skills. His defenses are typically disciplined and fundamentally sound, often playing a lot of zone coverage. On offense, his teams have been more conservative, often focusing on the running game."},
    {"team": "New York Giants", "coach": "Brian Daboll", "style": "Daboll is known for his offensive strategies and his work with quarterbacks. His offenses often feature a strong passing attack with a lot of variety in formations and personnel groupings. On defense, his teams have typically been aggressive, often blitzing to generate pressure."},
    {"team": "Philadelphia Eagles", "coach": "Nick Sirianni", "style": "Sirianni is an offensive-minded coach known for his energy and player development skills. His offenses often feature a balanced attack with a lot of pre-snap motion and misdirection. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "San Francisco 49ers", "coach": "Kyle Shanahan", "style": "Shanahan is known for his offensive genius, particularly his run-heavy schemes. His offenses often feature a lot of pre-snap motion and play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Seattle Seahawks", "coach": "Pete Carroll", "style": "Carroll's teams are known for their physical, run-first offense and aggressive, fast defense. His defensive philosophy is based on a 'bend but don't break' mentality, often giving up short gains but stiffening in the red zone."},
    {"team": "Tampa Bay Buccaneers", "coach": "Todd Bowles", "style": "Bowles is a defensive-minded coach known for his aggressive schemes. His defenses often feature a lot of blitzing to generate pressure. On offense, his teams have been more conservative, often focusing on the running game."},
    {"team": "Washington Commanders", "coach": "Ron Rivera", "style": "Rivera is known for his leadership and defensive strategies. His defenses are typically disciplined and fundamentally sound, often playing a lot of zone coverage. On offense, his teams have been more conservative, often focusing on the running game."}
]

print(len(afc_head_coaches))
print(len(nfc_head_coaches))

In [ ]:
import pandas as pd
from IPython.display import display, HTML

# Combine both AFC and NFC head coaches into one list
all_head_coaches = afc_head_coaches + nfc_head_coaches

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(all_head_coaches)

# Function to format the "style" column as multi-line text
def styled_display(row):
    return f"{row['style']}"

#     coach = f"<b>{row['coach']}</b>"
#     style = f"{row['style']}"
#     return f"{coach}<br>{style}"


# Apply the function to the "style" column to format it as multi-line text
df['style'] = df.apply(styled_display, axis=1)

# Set the option to display all rows and columns without truncation
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

# Display the DataFrame with HTML formatting
display(HTML(df.to_html(index=False)))

# Reset the option to the default values
pd.reset_option("display.max_rows")
pd.reset_option("display.max_columns")
pd.reset_option("display.width")


## Head Coach General Info

In [ ]:
## Scrape head coach names URL's

url = 'https://www.pro-football-reference.com/coaches/'  # Replace with your URL

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table containing the coaches
table = soup.find('table', {'id': 'coaches'})

# Initialize list to store coaches
coaches = []

# Loop through each row in the table
for row in table.find_all('tr'):
    # Initialize a dictionary to store coach data
    coach = {}

    # Extract the coach name and href link
    name_link = row.find('a')
    if name_link:
        coach['name'] = name_link.text
        coach['link'] = name_link['href']

    # Extract the number of super bowls
    super_bowls = row.find('td', {'data-stat': 'championships_super_bowl'})
    if super_bowls and super_bowls.text.strip():
        coach['super_bowls'] = int(super_bowls.text)

    # Extract the number of wins and losses
    wins = row.find('td', {'data-stat': 'wins'})
    if wins and wins.text.strip():
        coach['wins'] = int(wins.text)

    losses = row.find('td', {'data-stat': 'losses'})
    if losses and losses.text.strip():
        coach['losses'] = int(losses.text)

    # Add coach to list if data was found
    if coach:
        coaches.append(coach)
        
#print(f'There have been {coaches.count} head coaches in the NFL') 
print(f'There are {len(coaches)} coaches.')

for coach in coaches:
    print(coach)
        
# Create a DataFrame from the list of coaches
df = pd.DataFrame(coaches)

# Create new 'url' column
df['url'] = 'https://www.pro-football-reference.com' + df['link']

# Save the DataFrame to a CSV file
df.to_csv('./data/head_coaches.csv', index=False)


## Detailed Head Coach Data

In [ ]:
## Scrape data from individual head coach page

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

#url = "https://www.pro-football-reference.com/coaches/PaytSe0.htm"
df_url = pd.read_csv('./data/head_coaches.csv')
url = df_url['url'][3]
coach_name = df_url['name'][3] # Get coach name for file naming
coach_name_with_underscore = coach_name.replace(' ', '_')

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Find the div that contains the table
div = soup.find("div", {"id": "all_coaching_history"})

# The table is in a comment, so find the comment
comment = div.find(string=lambda text: isinstance(text, Comment))

# Parse the comment as HTML
table = BeautifulSoup(comment, "html.parser").find("table", {"id": "coaching_history"})

# Prepare list to store data
data = []

# Iterate through each row in the table (skipping the first header row)
for row in table.find_all("tr")[1:]:
    year_id = row.find("th", {"data-stat": "year_id"}).text.strip()
    coach_age = row.find("td", {"data-stat": "coach_age"}).text.strip()
    
    # Check if team name is a link or not
    employer_data = row.find("td", {"data-stat": "coach_employer"})
    if employer_data.a is not None:
        coach_employer = employer_data.a.text.strip()
    else:
        coach_employer = employer_data.text.strip()
        
    coach_role = row.find("td", {"data-stat": "coach_role"}).text.strip()
    
    ##### ADDING MORE MANUALLY
    
#<tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2016/">2016</a></th><td class="right " data-stat="age" >53</td><td class="left " data-stat="team" ><a href="/teams/gnb/2016.htm">GNB</a></td><td class="left " data-stat="league_id" >NFL</td><td class="right " data-stat="g" ><a href="/teams/gnb/2016_games.htm" title="Click for season gamelogs">16</a></td><td class="right " data-stat="wins" >10</td><td class="right " data-stat="losses" >6</td><td class="right iz" data-stat="ties" >0</td><td class="right " data-stat="win_loss_perc" >.625</td><td class="right " data-stat="srs_total" >2.8</td><td class="right " data-stat="srs_offense" >4.9</td><td class="right " data-stat="srs_defense" >-2.0</td><td class="right " data-stat="g_playoffs" >3</td><td class="right " data-stat="wins_playoffs" >2</td><td class="right " data-stat="losses_playoffs" >1</td><td class="right " data-stat="win_loss_playoffs_perc" >.667</td><td class="right " data-stat="rank_team" >1</td><td class="right " data-stat="chall_num" >5</td><td class="right " data-stat="chall_won" >3</td><td class="left " data-stat="coach_remarks" > </td></tr>
    wins = row.find("td", {"data-stat": "wins"}).text.strip()
    losses = row.find("td", {"data-stat": "losses"}).text.strip()
    ties = row.find("td", {"data-stat": "ties"}).text.strip()
    wins_playoffs = row.find("td", {"data-stat": "wins_playoffs"}).text.strip()
    losses_playoffs = row.find("td", {"data-stat": "losses_playoffs"}).text.strip()

    
    ##### ADDING MORE MANUALLY END
    
    
    data.append([year_id, coach_age, coach_employer, coach_role, wins, losses, ties, wins_playoffs, losses_playoffs])

# Convert list to DataFrame
df = pd.DataFrame(data, columns=["Year", "Age", "Employer", "Role", "Wins", "Losses", "Ties", "Wins_Playoffs", "Losses_Playoffs"])

# Write DataFrame to CSV
df.to_csv(f"./data/{coach_name_with_underscore}.csv", index=False)


In [4]:
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
# }

# Read the CSV file containing coach URLs
df_url = pd.read_csv('./data/head_coaches.csv')

# Loop through each row in the DataFrame
for index, row in df_url.iterrows():
    coach_name = row['name']
    coach_name_with_underscore = coach_name.replace(' ', '_')
    url = row['url']

    try:
#        response = requests.get(url, headers=headers)
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes

        soup = BeautifulSoup(response.text, "html.parser")

        # Find the div that contains the table
        div = soup.find("div", {"id": "all_coaching_history"})

        # The table is in a comment, so find the comment
        comment = div.find(string=lambda text: isinstance(text, Comment))

        # Parse the comment as HTML
        table = BeautifulSoup(comment, "html.parser").find("table", {"id": "coaching_history"})

        # Prepare list to store data
        data = []

        # Iterate through each row in the table (skipping the first header row)
        for row in table.find_all("tr")[1:]:
            year_id = row.find("th", {"data-stat": "year_id"}).text.strip()
            coach_age = row.find("td", {"data-stat": "coach_age"}).text.strip()

            # Check if team name is a link or not
            employer_data = row.find("td", {"data-stat": "coach_employer"})
            if employer_data.a is not None:
                coach_employer = employer_data.a.text.strip()
            else:
                coach_employer = employer_data.text.strip()

            coach_role = row.find("td", {"data-stat": "coach_role"}).text.strip()
            
            ##### ADDING MORE MANUALLY

            #<tr ><th scope="row" class="left " data-stat="year_id" ><a href="/years/2016/">2016</a></th><td class="right " data-stat="age" >53</td><td class="left " data-stat="team" ><a href="/teams/gnb/2016.htm">GNB</a></td><td class="left " data-stat="league_id" >NFL</td><td class="right " data-stat="g" ><a href="/teams/gnb/2016_games.htm" title="Click for season gamelogs">16</a></td><td class="right " data-stat="wins" >10</td><td class="right " data-stat="losses" >6</td><td class="right iz" data-stat="ties" >0</td><td class="right " data-stat="win_loss_perc" >.625</td><td class="right " data-stat="srs_total" >2.8</td><td class="right " data-stat="srs_offense" >4.9</td><td class="right " data-stat="srs_defense" >-2.0</td><td class="right " data-stat="g_playoffs" >3</td><td class="right " data-stat="wins_playoffs" >2</td><td class="right " data-stat="losses_playoffs" >1</td><td class="right " data-stat="win_loss_playoffs_perc" >.667</td><td class="right " data-stat="rank_team" >1</td><td class="right " data-stat="chall_num" >5</td><td class="right " data-stat="chall_won" >3</td><td class="left " data-stat="coach_remarks" > </td></tr>
            wins = row.find("td", {"data-stat": "wins"}).text
            losses = row.find("td", {"data-stat": "losses"}).text
            ties = row.find("td", {"data-stat": "ties"}).text
            wins_playoffs = row.find("td", {"data-stat": "wins_playoffs"}).text
            losses_playoffs = row.find("td", {"data-stat": "losses_playoffs"}).text


            ##### ADDING MORE MANUALLY END

            data.append([year_id, coach_age, coach_employer, coach_role, wins, losses, ties, wins_playoffs, losses_playoffs])

        # Convert list to DataFrame
        df = pd.DataFrame(data, columns=["Year", "Age", "Employer", "Role", "Wins", "Losses", "Ties", "Wins_Playoffs", "Losses_Playoffs"])

        # Write DataFrame to CSV with the modified coach's name as the file name
        file_name = f"./data/head_coaches_detailed/{coach_name_with_underscore}.csv"
        df.to_csv(file_name, index=False)

        print(f"CSV file '{file_name}' saved successfully.")

        # Pause for a short duration to avoid overwhelming the server
        time.sleep(5)  # Adjust the duration (in seconds) as needed

    except (requests.exceptions.RequestException, AttributeError, ValueError) as e:
        print(f"Error occurred while processing '{coach_name}': {str(e)}")


Error occurred while processing 'Don Shula': 'NoneType' object has no attribute 'text'
Error occurred while processing 'George Halas': 'NoneType' object has no attribute 'text'
Error occurred while processing 'Bill Belichick': 'NoneType' object has no attribute 'text'
Error occurred while processing 'Tom Landry': 'NoneType' object has no attribute 'text'
Error occurred while processing 'Andy Reid': 'NoneType' object has no attribute 'text'
Error occurred while processing 'Curly Lambeau': 'NoneType' object has no attribute 'text'
Error occurred while processing 'Paul Brown': 'NoneType' object has no attribute 'text'
Error occurred while processing 'Marty Schottenheimer': 'NoneType' object has no attribute 'text'
Error occurred while processing 'Chuck Noll': 'NoneType' object has no attribute 'text'
Error occurred while processing 'Dan Reeves': 'NoneType' object has no attribute 'text'
Error occurred while processing 'Chuck Knox': 'NoneType' object has no attribute 'text'
Error occurred 

KeyboardInterrupt: 

In [ ]:
## Download each html file instead
## Alt method

# # Load the CSV file into a pandas DataFrame
# csv_file = './data/head_coaches.csv'  # Replace this with the path to your CSV file
# df = pd.read_csv(csv_file)

# # Create the 'output' folder if it doesn't exist
# os.makedirs('output', exist_ok=True)

# # Loop through the 'url' column and download HTML for each URL
# for url in df['url']:
#     try:
#         response = requests.get(url)
#         response.raise_for_status()  # Check for any errors

#         # Extract the filename from the URL
#         filename = os.path.join('output', url.split('/')[-1] + '.html')

#         # Save the HTML content to a file
#         with open(filename, 'wb') as f:
#             f.write(response.content)

#         print(f"Downloaded HTML from {url} and saved as {filename}")
        
#         # Introduce a delay to avoid rate-limiting
#         time.sleep(3)  # Wait for 5 seconds before the next request

#     except Exception as e:
#         print(f"Error downloading {url}: {e}")
